In [1]:
%pip install azure-search-documents==11.6.0b9 azure-identity python-dotenv pandas jinja2 --quiet

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os

AZURE_SEARCH_SERVICE_ENDPOINT=os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
AZURE_SEARCH_INDEX=os.getenv("AZURE_SEARCH_INDEX_NAME")
AZURE_SEARCH_ADMIN_KEY=os.getenv("AZURE_SEARCH_ADMIN_KEY")